In [2]:
import boto3
import pandas as pd

s3 = boto3.client('s3')

print(s3)

In [2]:
response = s3.list_buckets()

print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  cyclistdata
  streaming-analytics-gra
  us-pills-usage


In [2]:
import boto3
import pandas as pd
import io

def pd_read_s3_parquet(key, bucket, s3_client=None, **args):
    if s3_client is None:
        s3_client = boto3.client('s3')
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    return pd.read_parquet(io.BytesIO(obj['Body'].read()), **args)

def pd_read_s3_multiple_parquets(filepath, bucket, s3=None, 
                                 s3_client=None, verbose=False, **args):
    if not filepath.endswith('/'):
        filepath = filepath + '/'  # Add '/' to the end
    if s3_client is None:
        s3_client = boto3.client('s3')
    if s3 is None:
        s3 = boto3.resource('s3')
    s3_keys = [item.key for item in s3.Bucket(bucket).objects.filter(Prefix=filepath)
               if item.key.endswith('.parquet')]
    if not s3_keys:
        print('No parquet found in', bucket, filepath)
    elif verbose:
        print('Load parquets:')
        for p in s3_keys: 
            print(p)
    dfs = [pd_read_s3_parquet(key, bucket=bucket, s3_client=s3_client, **args) 
           for key in s3_keys]
    return pd.concat(dfs, ignore_index=True)

In [3]:
df = pd_read_s3_multiple_parquets('ratings2022/01/08/06/', 'captains-rating')

print(df.head())

                user     favoritecaptain  rating               timestamp
0     Joseph Compton       Turanga Leela     845 2022-01-08 06:23:33.536
1   Kimberly Edwards    Malcolm Reynolds      31 2022-01-08 06:23:33.727
2        Maria Hayes         Jacob Keyes     583 2022-01-08 06:23:33.772
3     Troy Gutierrez    Christopher Pike     698 2022-01-08 06:23:33.811
4  Brandon Patterson  Khan Noonien Singh     875 2022-01-08 06:23:33.850
